In [0]:
pip install SALib

In [0]:
from SALib.analyze import morris
from SALib.util import read_param_file
from SALib.plotting.morris import horizontal_bar_plot, covariance_plot, \
    sample_histograms
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.pyplot as plt
from google.colab import files

def fullprint(*args, **kwargs):
  from pprint import pprint
  import numpy
  opt = numpy.get_printoptions()
  numpy.set_printoptions(threshold=numpy.inf)
  pprint(*args, **kwargs)
  numpy.set_printoptions(**opt)

In [0]:
#IMPORT DATA
problem = read_param_file('/content/drive/My Drive/Colab Notebooks/Integrated Design/2166/parameter_file.txt')
#model_input = np.loadtxt('/content/drive/My Drive/Colab Notebooks/Integrated Design/model_input.txt')
#model_output = np.loadtxt('/content/drive/My Drive/Colab Notebooks/Integrated Design/model_output.txt')
data = np.loadtxt('/content/drive/My Drive/Colab Notebooks/Integrated Design/2166/2166.txt')
print("data ndim: ", data.ndim)
print("data shape:", data.shape)
print("data size: ", data.size)

In [0]:
for i in [1,2,3,4,5,6,7,8,9,10,12,13,16,17]:
  print(data[data[:,13]==i,0:13].shape)

In [0]:
D=problem["num_vars"] #number of variables
for x in [0,1]: #x=0 kgCO2, x=1 %comf_t
  for HVACsys in [1,2,3,4,5,6,7,8,9,10,12,13,16,17]:
    
    # Prepare data for the analysis for each system
    filename = 'HVACsys'+str(HVACsys)
    rows=data[data[:,13]==HVACsys,0:13].shape[0] #number of inputs/outputs
    n=divmod(rows,14)[0]
    model_input=data[data[:,13]==HVACsys,0:13][0:n*(D+1),:]
    model_output=data[data[:,13]==HVACsys,14+x:15+x][0:n*(D+1),:]
    print(filename)
    # Perform the sensitivity analysis
    Si = morris.analyze(problem, model_input, model_output, conf_level=0.95,
                        print_to_console=True)
    #create plots
    fig1, (ax1, ax2) = plt.subplots(1, 2)
    ax1.set_title(filename)
    ax2.set_title(filename)
    if x == 0:
      horizontal_bar_plot(ax1, Si, {}, sortby='mu_star', unit=r"kgCO2")
      covariance_plot(ax2, Si, {}, unit=r"kgCO2")
      filename_update=filename+"_kgCO2"
    else:
      horizontal_bar_plot(ax1, Si, {}, sortby='mu_star', unit=r"%ComfTime")
      covariance_plot(ax2, Si, {}, unit=r"%ComfTime")
      filename_update=filename+"_ComfTime"
    #save and download plots  
    plt.savefig(filename_update)
    files.download(filename_update+".png")
    plt.close()